This code is used to infer and save the intents of 3986 trajectory samples of encountering ships. 

In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()


  Activating project at `~/Documents/intent_inference/codes`


In [3]:
using ReactiveMP
using GraphPPL
using Rocket
using Distributions
using Random
using BenchmarkTools
using LinearAlgebra

In [4]:

using CSV, DataFrames
df = CSV.File("/Users/jia/Documents/intent_inference/data/prior07256f.csv", header=false) |> DataFrame


,Column1,Column2,Column3,Column4,Column5,Column6
,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0043624,0.0136669,1.78e-6,6.3e-7,6.3e-7,3.57e-5
2,8.21e-6,0.0137051,8.31e-7,2.52e-7,2.52e-7,3.52e-5
3,0.00436199,0.0136642,1.67e-6,-1.06e-6,-1.06e-6,3.7e-5
4,-0.00436012,0.000554796,2.91e-6,8.28e-7,8.28e-7,5.44e-5
5,-1.61e-5,-0.000208031,6.29e-7,-3.6e-7,-3.6e-7,5.19e-5
6,0.00436274,0.00152798,2.02e-6,-1.18e-6,-1.18e-6,5.68e-5
7,-0.00436173,-0.0136781,2.72e-6,2.38e-7,2.38e-7,3.37e-5
8,1.98e-5,-0.0136899,8.57e-7,4.43e-7,4.43e-7,3.72e-5
9,0.00436055,-0.0136715,1.44e-6,1.14e-6,1.14e-6,3.56e-5


In [5]:
# course_=1/180*π
# acc=0.5144444

course_=1.0
acc=1.0
prior_m1=[0.0, 0.0,df[1,1]*course_,df[1,2]*acc]
prior_m2=[0.0, 0.0,df[2,1]*course_,df[2,2]*acc]
prior_m3=[0.0, 0.0,df[3,1]*course_,df[3,2]*acc]
prior_m4=[0.0, 0.0,df[4,1]*course_,df[4,2]*acc]
prior_m5=[0.0, 0.0,df[5,1]*course_,df[5,2]*acc]
prior_m6=[0.0, 0.0,df[6,1]*course_,df[6,2]*acc]
prior_m7=[0.0, 0.0,df[7,1]*course_,df[7,2]*acc]
prior_m8=[0.0, 0.0,df[8,1]*course_,df[8,2]*acc]
prior_m9=[0.0, 0.0,df[9,1]*course_,df[9,2]*acc]
prior_w1=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[1,3]*course_*course_ df[1,4]*course_*acc; 0.0 0.0 df[1,5]*course_*acc df[1,6]*acc*acc]
prior_w2=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[2,3]*course_*course_ df[2,4]*course_*acc; 0.0 0.0 df[2,5]*course_*acc df[2,6]*acc*acc]
prior_w3=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[3,3]*course_*course_ df[3,4]*course_*acc; 0.0 0.0 df[3,5]*course_*acc df[3,6]*acc*acc]
prior_w4=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[4,3]*course_*course_ df[4,4]*course_*acc; 0.0 0.0 df[4,5]*course_*acc df[4,6]*acc*acc]
prior_w5=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[5,3]*course_*course_ df[5,4]*course_*acc; 0.0 0.0 df[5,5]*course_*acc df[5,6]*acc*acc]
prior_w6=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[6,3]*course_*course_ df[6,4]*course_*acc; 0.0 0.0 df[6,5]*course_*acc df[6,6]*acc*acc]
prior_w7=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[7,3]*course_*course_ df[7,4]*course_*acc; 0.0 0.0 df[7,5]*course_*acc df[7,6]*acc*acc]
prior_w8=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[8,3]*course_*course_ df[8,4]*course_*acc; 0.0 0.0 df[8,5]*course_*acc df[8,6]*acc*acc]
prior_w9=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 df[9,3]*course_*course_ df[9,4]*course_*acc; 0.0 0.0 df[9,5]*course_*acc df[9,6]*acc*acc]

4×4 Matrix{Float64}:
 1.0  0.0  0.0      0.0
 0.0  1.0  0.0      0.0
 0.0  0.0  1.44e-6  1.14e-6
 0.0  0.0  1.14e-6  3.56e-5

In [6]:
@model function filtering(Q::ConstVariable, R::ConstVariable)
    # δt=15
    
    A = datavar(Vector{Float64})
    B = constvar([ .0 .0 .0 .0; .0 .0 .0 .0; .0 .0 1. .0; .0 .0 .0 1. ]*δt) 
    # prior_m1=[ 0.0, 0.0, 1/8π, 0.01 ]
    # prior_m2=[ 0.0, 0.0, -1/8π, 0.01 ]
    # prior_m3=[ 0.0, 0.0, 0.0, 0.01 ]
    # prior_w1=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1/16π .0; 0.0 0.0 0.0 0.05]
    # prior_w2=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1/16π .0; 0.0 0.0 0.0 0.05]
    # prior_w3=[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1/16π .0; 0.0 0.0 0.0 0.05]

    nmixtures=9
    
    m = randomvar(nmixtures)
    w = randomvar(nmixtures)

    mean_mean_cov       = [ 1e-9 0.0 0.0 0.0; 0.0 1e-9 0.0 0.0; 0.0 0.0 1e-6 0.0; 0.0 0.0 0.0 1e-6 ]

    m[1] ~ MvNormalMeanCovariance(prior_m1, mean_mean_cov)
    m[2] ~ MvNormalMeanCovariance(prior_m2, mean_mean_cov)
    m[3] ~ MvNormalMeanCovariance(prior_m3, mean_mean_cov)
    m[4] ~ MvNormalMeanCovariance(prior_m4, mean_mean_cov)
    m[5] ~ MvNormalMeanCovariance(prior_m5, mean_mean_cov)
    m[6] ~ MvNormalMeanCovariance(prior_m6, mean_mean_cov)
    m[7] ~ MvNormalMeanCovariance(prior_m7, mean_mean_cov)
    m[8] ~ MvNormalMeanCovariance(prior_m8, mean_mean_cov)
    m[9] ~ MvNormalMeanCovariance(prior_m9, mean_mean_cov)



    w[1] ~ Wishart(10, prior_w1)
    w[2] ~ Wishart(10, prior_w2)
    w[3] ~ Wishart(10, prior_w3)
    w[4] ~ Wishart(10, prior_w4)
    w[5] ~ Wishart(10, prior_w5)
    w[6] ~ Wishart(10, prior_w6)
    w[7] ~ Wishart(10, prior_w7)
    w[8] ~ Wishart(10, prior_w8)
    w[9] ~ Wishart(10, prior_w9)

    means = tuple(m...)
    precs = tuple(w...)
    # control ~ NormalMixture(intent, means, precs)

    x_k_mean = datavar(Vector{Float64})
    x_k_cov  = datavar(Matrix{Float64})
    
    #control depends on intent, and intent is a category Distributions.
    #control is GMM Distributions

    intent ~ Categorical([1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9]) where { q = MeanField() }
    println(intent)
    # println((m1, m2))
    control = randomvar()
    control ~ NormalMixture(intent, means, precs) where { q = MeanField() }

    # control ~ MvNormalMeanCovariance(control_mean,  control_covariance)
    x_k ~ MvNormalMeanCovariance(x_k_mean, x_k_cov)
    
    tmp1 ~ (A + x_k) 
    tmp2 ~ (B * control)

    x_k_p_one ~ MvNormalMeanCovariance(tmp1 + tmp2, Q)

    # own ship state
    o_x_k_p_one=datavar(Vector{Float64})
    
    # relative observation between two ships
    y_k = datavar(Vector{Float64})
    
    y_k ~ MvNormalMeanCovariance(x_k_p_one-o_x_k_p_one, R)
    
    return x_k_mean, x_k_cov, x_k_p_one, y_k, A, control, intent, m, w,o_x_k_p_one
end

In [7]:
function getAFromState(x,δt)
    A=zeros(4)
    A[1]=x[4]*sin(x[3])*δt
    A[2]=x[4]*cos(x[3])*δt
    return A
end

getAFromState (generic function with 1 method)

In [8]:
function inference1(δt, max_data_points, max_vmp_iterations,mode,target_ship,own_ship)
    nmixtures=9
    data_stream1 = ReplaySubject(Vector{Float64}, 1)
    data_stream2 = ReplaySubject(Vector{Float64}, 1)

    saved_controls = []
    saved_position = []
    saved_observations = []
    saved_intent   = []
    saved_own_position=[]
    ii = 0

    # y_current = zeros(4)
    # y_current = [0.0,0.0,0,1.0]
    
    # TODO: Depend on δt
    Q = diageye(4)*1.0
    R = diageye(4)*10
    
    model, (x_k_mean, x_k_cov, x_k_p_one, y_k, A, control, intent, m, w, o_x_k_p_one) = filtering(Q, R)
    # ** setmarginal

    setmarginal!(m[1], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[2], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[3], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[4], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[5], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[6], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[7], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[8], MvNormalMeanCovariance(zeros(4), diageye(4)))
    setmarginal!(m[9], MvNormalMeanCovariance(zeros(4), diageye(4)))
    

    setmarginal!(w[1], Wishart(10, diageye(4)))
    setmarginal!(w[2], Wishart(10, diageye(4)))
    setmarginal!(w[3], Wishart(10, diageye(4)))
    setmarginal!(w[4], Wishart(10, diageye(4)))
    setmarginal!(w[5], Wishart(10, diageye(4)))
    setmarginal!(w[6], Wishart(10, diageye(4)))
    setmarginal!(w[7], Wishart(10, diageye(4)))
    setmarginal!(w[8], Wishart(10, diageye(4)))
    setmarginal!(w[9], Wishart(10, diageye(4)))

    setmarginal!(intent, vague(Categorical, 9))
    setmarginal!(control, vague(MvNormalMeanCovariance, 4))
    setmarginal!(x_k_p_one, vague(MvNormalMeanCovariance, 4))
    

    

    stream = combineLatest(
        (
        getmarginal(x_k_p_one, IncludeAll()), 
        getmarginal(control, IncludeAll()), 
        getmarginal(intent, IncludeAll()), 
        data_stream1,
        data_stream2
        ), PushNew()
    )


    fe_actor = ScoreActor(Float64)

    fe_subscription = subscribe!(score(Float64, BetheFreeEnergy(), model), fe_actor)

    finish = Condition()
    
    subscription = subscribe!(stream |> async() |> take(max_data_points) |> tap_on_complete(() -> notify(finish)), (d) -> begin

        x_k_p_one_posterior = d[1]
        control_posterior   = d[2] 
        intent_posterior    = d[3]
        data_point1         = d[4]
        data_point2         = d[5]

        push!(saved_observations, data_point1)

        μ, Σ = mean_cov(x_k_p_one_posterior)
        pA = getAFromState(μ, δt)
        
        for k in 1:max_vmp_iterations
            update!(x_k_mean, μ)
                
            update!(x_k_cov, Σ)
                
            # Update A here
            update!(A, pA)

            update!(o_x_k_p_one, data_point2)
            update!(y_k, data_point1)
            # update!(intent, mean(intent_posterior))
        end

        release!(fe_actor)

        index = length(saved_controls)
        ii = ii + 1

        # Save our result

        push!(saved_controls, Rocket.getrecent(getmarginal(control, IncludeAll())))   
        push!(saved_position, Rocket.getrecent(getmarginal(x_k_p_one, IncludeAll())))   
        push!(saved_intent, Rocket.getrecent(getmarginal(intent, IncludeAll())))   

            
        # update y_current somehow

        #true action
        
        c_m=zeros(4)
        c_m[3]=0.0
        c_m[4]=0.0

        # that's for simulation
        # y_own=own_ship_moving(data_point2,δt,mode)
        # Welcome to new world
        y_own=own_ship[ii,:]
        
        push!(saved_own_position,y_own)
        next!(data_stream2, y_own)

        # that's for simulation
        # y_next = generateNewObservation(ii, δt, data_point1+data_point2, c_m,mode)  
        # Welcome to new world
        y_next=target_ship[ii,:]

        y_relative=y_next-y_own


        
            
        next!(data_stream1, y_relative)

    end)

    # Start to initial
    if mode=="head-on"
        y_initial=[10.0;10.0 ;π/2.001;1.0]
        y_own_initial=[0.0;800.0 ;-π/2;1.0]
    end
    if mode=="overtaking"
        y_initial=[10.0;10.0 ;π/2.001;0.8]
        y_own_initial=[-20.0;100.0;π/2.00001;1.0]
    end

    if mode=="crossing"
        y_initial=[-10.0;10.0 ;π/2.001;2.0]
        y_own_initial=[-700.0;410.0;0.00001;5.0]
    end
    next!(data_stream2, y_own_initial) # Initial state
    next!(data_stream1, y_initial-y_own_initial) # Initial state

    
    wait(finish)
    
    return saved_controls, saved_observations, saved_position, saved_intent, getvalues(fe_actor),saved_own_position
end

inference1 (generic function with 1 method)

In [9]:
δt=15

15

In [10]:
function score2prob(intent_score)
    list_i=[]
    T=1000
    for ii in intent_score
        v_list=[]
        for v in ii
            v=exp(v*T)
            push!(v_list,v)
        end
        vv_list=[]
        for vv in v_list
            vv=vv/sum(v_list)
            push!(vv_list,vv)
        end
        push!(list_i,vv_list)
    end
    return list_i;
end
function find_max(intent_score)
    res_list=[]
    for ii in intent_score
        _,res=findmax(ii)
        push!(res_list,res)
    end
return res_list;
end

function get_prob(c)
    p1=logpdf(d1, c)
    p2=logpdf(d2, c)
    p3=logpdf(d3, c)
    p4=logpdf(d4, c)
    p5=logpdf(d5, c)
    p6=logpdf(d6, c)
    p7=logpdf(d7, c)
    p8=logpdf(d8, c)
    p9=logpdf(d9, c)
    pp=[p1,p2,p3,p4,p5,p6,p7,p8,p9];
    return pp;
end
function  accuracy_calculate(list1,list2)
    sum=0
    for i in 1:length(list1)
        if list1[i]==list2[i]
            sum=sum+1
        end
    end
    return sum/length(list1)
end

accuracy_calculate (generic function with 1 method)

In [11]:
function eliminate_course_trans(cmatrix)
    res=[]
    ii=0
    for mm in cmatrix
        ii=ii+1
        if mm[3]*δt>π
            mm[3]=(2*π-mm[3]*δt)/δt
        end
        if mm[3]*δt<-π
            mm[3]=(-2*π-mm[3]*δt)/δt
        end
        push!(res,mm)
    end
    return res
end

eliminate_course_trans (generic function with 1 method)

In [12]:
function backed_get_prob(ex)
    middle=[]
    res=[]
    T=3
    for ee in ex
        ee=exp(ee*T)
        push!(middle,ee)
    
    end
    for ee in middle
        ee=ee/sum(middle)
        push!(res,ee)
    end
    return res
end

backed_get_prob (generic function with 1 method)

In [14]:
using Plots
# mf="/Users/jia/Documents/intent_inference/data/xx/"
mf="/Users/jia/Documents/intent_inference/data/test_data3/"
mydir = readdir(mf)
file_list=[]
for f in mydir
        push!(file_list,string(mf,f))
end
using CSV, DataFrames
δt=15
# Own=true
Own=false

# f=file_list[2]
f="/Users/jia/Documents/intent_inference/data/test_data//1543593600_1543597185_412436640_413521690.csv"
save_plot="./origin_plot/"
# for index=2:length(file_list)
mean_mean_cov= [ 1e-9 0.0 0.0 0.0; 0.0 1e-9 0.0 0.0; 0.0 0.0 1e-6 0.0; 0.0 0.0 0.0 1e-6 ];
d1 = MvNormalMeanCovariance(prior_m1, prior_w1);
d2 = MvNormalMeanCovariance(prior_m2, prior_w2);
d3 = MvNormalMeanCovariance(prior_m3, prior_w3);
d4 = MvNormalMeanCovariance(prior_m4, prior_w4);
d5 = MvNormalMeanCovariance(prior_m5, prior_w5);
d6 = MvNormalMeanCovariance(prior_m6, prior_w6);
d7 = MvNormalMeanCovariance(prior_m7, prior_w7);
d8 = MvNormalMeanCovariance(prior_m8, prior_w8);
d9 = MvNormalMeanCovariance(prior_m9, prior_w9);
for index=2:length(file_list)
    f=file_list[index]
    print(f)
    # for f in file_list
    
    if true
        df = CSV.File(f, header=false) |> DataFrame
        # print(df)
        # println(df[:,1])
        own_x=df[:,13]*1000
        own_y=df[:,14]*1000
        own_ϕ=df[:,4]/180*π
        own_v=df[:,3]*0.514444
        own_ship=[own_x own_y own_ϕ own_v]
        # oo=own_ship
        
        target_x=df[:,15]*1000
        target_y=df[:,16]*1000
        target_ϕ=df[:,10]/180*π
        target_v=df[:,9]*0.514444
        target_ship=[target_x target_y target_ϕ target_v]
        # println(target_ship)

        # break
    end
    if ! Own
        own_ship,target_ship=target_ship,own_ship
    end
    rng= MersenneTwister(1234)
    mode="crossing"
    controls, obs, pos,intents, fe ,own_ship_output= inference1(δt, trunc(Int,length(target_ship)/4), 15,mode,target_ship,own_ship);
    c_matix=mean.(controls);
    c_matix=eliminate_course_trans(c_matix)
    ress=[]
    for ci=1:length(c_matix);
        pci=get_prob(c_matix[ci]);
        _,m=findmax(pci);
        push!(ress,m);
    end
    # print(ress[118:150])
    ress=ress[3:end]
    # println(ress)
    if Own
        gd=df[2:end,18]
        fw=open(string("./res8/own_res_",split(f,"/")[end]),"w")
    else
        gd=df[2:end,17]
        fw=open(string("./res8/tar_res_",split(f,"/")[end]),"w")
    end
    # println(gd)

    
    for ee =1:length(ress)
        write(fw,string(ress[ee],",",string(gd[ee])))
        write(fw,"\n")
    end
    close(fw)
    
end
# p1 = scatter(own_ship[:,1], own_ship[:,2],  ms = 2)

# p1 = scatter(map(o -> o[1], o_w), map(o -> o[2], o_w),  ms = 2, xlims=[-800, 800],ylims=[0, 800])

# plot(o_w, label = "c[3]")


/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_413408630.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_413505980.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_413521690.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_413552920.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_413699000.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_413706710.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_412436640_414185000.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543593600_1543597185_413376840_414185000.csvRandomVariable(intent)
/Users/jia/Documents/int

Excessive output truncated after 524348 bytes.

intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896600_1543900185_414131000_414783000.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412439190_413696360.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412439190_414131000.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412439190_414783000.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412499000_412760760.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412499000_413431120.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412514000_412760760.csvRandomVariable(intent)
/Users/jia/Documents/intent_inference/data/test_data3/1543896900_1543900485_412514000_413431120.csvRandomVariable(intent)
/Users/jia/Docum

In [15]:
# excel_=get_prob(c_matix[113])
excel_=get_prob(c_matix[131])
e=backed_get_prob(excel_)
f=open("res131.csv","w")
for ee in e
    write(f,string(ee))
    write(f,"\n")
end
close(f)


LoadError: UndefVarError: c_matix not defined

In [ ]:
# print(df[:,18])
if Own
    findall(x -> x == 7.0,df[:,18])
else
    findall(x -> x == 7.0,df[:,17])
end

In [ ]:
plot(ress[2:length(ress)], label = "Predicted intent")
if Own
    plot!(df[2:end,18],label="Ground truth",ylim=[1,9])

else
    plot!(df[2:end,17],label="Ground truth")
end

In [ ]:
accuracy_calculate(ress[2:length(ress)],df[:,18])

In [ ]:
cc= mean.(controls)[126]/15
# cc=[0.0, 0.0, -0.003011298479122978, 0.010784568172342095]
print(cc)
get_prob(cc)

In [ ]:
plot(map(c -> c[3] , mean.(controls)[3:150]), label = "Turning rate")
plot!(turning_target/ δt,xlims=[0,150])
# plot(map(c -> c[3] / δt, mean.(controls)), ribbon = map(c -> c[3] / δt, var.(controls)), label = "c[3]")
# plot!(map(c -> c[4] / δt, mean.(controls)), label = "Acceleration")
# plot!(map(c -> c[3] / δt, var.(controls)), label = "c[3]")
# plot!(map(c -> c[4] / δt, var.(controls)), label = "c[4]v")


In [ ]:
plot(accleration_target/ δt,xlims=[0,151])
plot!(map(c -> c[4] , mean.(controls)[3:150]), label = "Acceleration")

In [ ]:
# println(mean.(intents))

In [ ]:
plot(mean.(intents),label = "Intent")
# println(var.(intents))

In [ ]:
cov.(controls)[2]